In [ ]:
import pandas as pd
import os
from tqdm import tqdm
from openai import OpenAI

# Solar API 설정
UPSTAGE_API_KEY = ""  # 발급받은 API 키
client = OpenAI(
    api_key=UPSTAGE_API_KEY,
    base_url="https://api.upstage.ai/v1/solar"
)

# Few-shot 예제
few_shot_examples = [
    {
        "dialogue": "#Person1#: 여름이 다 되어간다는 게 믿기지 않아.\n"
                    "#Person2#: 응, 알아. 이번 해는 정말 빨리 갔어.\n"
                    "#Person1#: 이번 여름 휴가에 뭐 할 거야?\n"
                    "#Person2#: 나는 회사에서 일할 거야.\n"
                    "#Person1#: 네가 요리를 할 줄 몰랐어.\n"
                    "#Person2#: 나는 그저 조수일 뿐이야.\n"
                    "#Person1#: 꽤 쉬워 보이네.\n"
                    "#Person2#: 그건 일부일 뿐이야. 파티 도중에는 손님들에게 음식과 음료를 제공해야 해.",
        "summary": "#Person2#는 #Person1#에게 여름 휴가 동안 파티를 도와주는 회사에서 일할 것이라고 말한다. #Person1#는 그것이 멋진 직업이라고 생각한다."
    },
    {
        "dialogue": "#Person1#: 인터넷에서 빌 게이츠의 집을 본 적이 있나요?\n"
                    "#Person2#: 아니요. 어떤 모습인가요?\n"
                    "#Person1#: 그 집에는 도서관, 극장, 수영장, 그리고 게스트 하우스가 있어요. 정말 놀라운 집이에요!",
        "summary": "#Person1#은 #Person2#에게 빌 게이츠의 집이 어떤지 설명합니다. 그 집에는 도서관, 극장, 수영장, 그리고 게스트 하우스가 있습니다."
    },
    {
        "dialogue": "#Person1#: 지난달에 새로운 일자리가 30만 개 이상 늘어났습니다. 이는 3년간의 침체기 후 일자리 시장의 개선을 보여줍니다.\n"
                    "#Person2#: 음식 서비스, 건강 관리, 서비스 등 많은 산업이 긍정적인 성장을 경험하고 있습니다.",
        "summary": "#Person1#은 #Person2#와 지난달 새로운 일자리 창출에 대해 이야기하며, 긍정적인 성장 추세를 언급합니다."
    }
]



# 프롬프트 생성 함수
def build_prompt_with_examples(dialogue):
    system_prompt = (
    "You are an expert in summarizing Korean dialogues. Generate a concise and accurate summary in Korean, "
    "optimized for ROUGE-1, ROUGE-2, and ROUGE-L metrics. Ensure the summary includes the main points of the dialogue, "
    "avoids unnecessary repetition, and is written in a formal yet natural style suitable for documentation. "
    "The summary should be concise, with a length approximately between 60 and 100 characters, "
    "matching the statistical distribution of 80 characters on average. Avoid exceeding 120 characters for clarity. "
    "Preserve the flow and key context of the dialogue. For example, convert casual phrases into formal equivalents while maintaining readability."
    )


    examples = ""
    for i, example in enumerate(few_shot_examples):
        examples += f"Example {i + 1} Dialogue:\n{example['dialogue']}\nSummary:\n{example['summary']}\n\n"

    user_prompt = f"{examples}New Dialogue:\n{dialogue}\nSummary:\n"
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]

# Summarization 함수
def summarize_with_solar(dialogue):
    response = client.chat.completions.create(
        model="solar-1-mini-chat",
        messages=build_prompt_with_examples(dialogue),
        temperature=0.5,
        top_p=0.9,
    )
    # 요약 내용 가져오기
    generated_summary = response.choices[0].message.content.strip()

    # "요약:" 이후 중복된 텍스트 제거
    if "요약:" in generated_summary:
        parts = generated_summary.split("요약:", 1)
        generated_summary = parts[0].strip()  # "요약:" 이전 부분만 사용

    return generated_summary

# 데이터 증강 함수
def generate_augmented_data(input_csv, output_csv, scale_factor=1.2):
    df = pd.read_csv(input_csv)
    original_len = len(df)
    target_len = int(original_len * scale_factor)

    print(f"Original dataset size: {original_len}")
    print(f"Target augmented dataset size: {target_len}")

    # 증강 데이터 생성
    augmented_data = []
    for idx in tqdm(range(target_len - original_len)):
        try:
            # 원본 데이터 중 임의 샘플 선택
            sample = df.sample(1).iloc[0]
            dialogue = sample['dialogue']
            new_summary = summarize_with_solar(dialogue)  # Solar API를 이용한 요약

            # 새 데이터 추가
            augmented_data.append({"dialogue": dialogue, "summary": new_summary})
        except Exception as e:
            print(f"Error at index {idx}: {e}")
            continue

    # 기존 데이터에 증강 데이터 추가
    augmented_df = pd.concat([df, pd.DataFrame(augmented_data)], ignore_index=True)
    augmented_df.to_csv(output_csv, index=False)
    print(f"Augmented dataset saved to {output_csv}")

# 실행 코드
if __name__ == "__main__":
    DATA_PATH = "../data"  # train.csv 파일 경로
    OUTPUT_PATH = "../data"  # augmented_train.csv 저장 경로

    train_file = os.path.join(DATA_PATH, "train.csv")
    augmented_train_file = os.path.join(OUTPUT_PATH, "augmented_train_06.csv")

    # Augmented Dataset 생성
    generate_augmented_data(train_file, augmented_train_file)

    print("Augmented dataset generation complete!")


Original dataset size: 12457
Target augmented dataset size: 14948


100%|██████████| 2491/2491 [46:20<00:00,  1.12s/it]

Augmented dataset saved to ../data/augmented_train_06.csv
Augmented dataset generation complete!
